In [1]:
import psycopg2
from psycopg2 import sql
from psycopg2 import Error

In [2]:
DB_HOST = 'pavlov.ing.puc.cl'
DB_PORT = '5432'
DB_USER = 'grupo8'
DB_PASSWORD = 'bulla8'
DB_NAME = 'grupo8e2'

In [26]:
#Muestre todos los restaurantes que ofrezcan un plato específico (string) ingresado y que esté disponible
def consulta1(): 
    #query devuelve nombre y sucursal de los restaurantes que ofrecen el plato ingresado
    query = """SELECT Restaurant.id, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.nombre = %(nombre)s 
        AND Plato.disponibilidad = TRUE 
        AND Restaurant.vigente = TRUE"""
    return query

#Muestre todos los pedidos de un usuario específico ingresado (email string) y su gasto mensual (solo los pedidos concretados)
def consulta2():
    query = """SELECT * 
        FROM Pedido
        JOIN Usuario ON Pedido.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s
        AND Pedido.fecha BETWEEN %(fecha_inicio)s AND %(fecha_fin)s"""
    
    query1 = """ SELECT Usuario.email,  (SUM(Empresa.precio_mensual) + SUM(Empresa.precio_anual / 12) + gasto_mensual_sin_suscripcion.gasto_mensual) as gasto_mensual 
            FROM Usuario, Suscripcion, Empresa, (SELECT Usuario.email as email_usuario, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_mensual
                                        FROM Usuario, Suscripcion, Pedido,
                                        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * Plato.cantidad) as precio_comida 
                                            FROM Pedido
                                            JOIN Despacho ON Despacho.pedido_id = Pedido.id
                                            JOIN Empresa ON Despacho.empresa_id = Empresa.id
                                            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                                            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                                            WHERE Pedido.estado = 'entregado a cliente' 
                                            AND Pedido.fecha BETWEEN %(fecha_inicio)s AND %(fecha_fin)s
                                            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido_mes) 
                                        WHERE Usuario.id = gasto_pedido.usuario_id 
                                        AND Pedido.id = gasto_pedido.pedido_id 
                                        AND Suscripcion.usuario_id = Usuario.id
                                        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                                                        FROM Suscripcion 
                                                                        WHERE Suscripcion.usuario_id = Usuario.id) 
                                                                        AND Suscripcion.estado = 'Vigente') 
                                        GROUP BY Usuario.email) as gasto_mensual_sin_suscripcion 
        WHERE gasto_mensual_sin_suscripcion.email_usuario = Usuario.email 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id = Empresa.id 
        AND Suscripcion.estado = 'Vigente' 
        AND Suscripcion.tipo = 'mensual' 
        GROUP BY Usuario.email"""

    return query, query1

#Muestre todos pedidos concretados y cancelados y el valor total de ellos
def consulta3():
    query = """SELECT Pedido.id, Usuario.nombre, Usuario.email, Pedido.estado, (SUM(gasto_pedido_comida.precio_comida) + SUM(gasto_pedido_comida.precio_despacho)) as gasto_pedido, gasto_pedido_comida.precio_comida, gasto_pedido_comida.precio_despacho
            FROM Pedido
            JOIN (SELECT DISTINCT Pedido.id as pedido_id, Usuario.nombre, Usuario.email,
                    CASE
                        WHEN (Suscripcion.ciclo = 'anual' AND Suscripcion.fecha_ultimo_pago >= (NOW() - INTERVAL '1' YEAR) AND Empresa.vigente = TRUE)  
                        OR  (Suscripcion.ciclo = 'mensual' AND Suscripcion.fecha_ultimo_pago >= (NOW() - INTERVAL '1' MONTH) AND Empresa.vigente = TRUE)
                        THEN 0
                        ELSE Empresa.precio_despacho
                    END AS precio_despacho,
                    SUM(Plato.precio * ContenidoPedido.cantidad) as precio_comida
                FROM Pedido
                JOIN Despacho ON Despacho.pedido_id = Pedido.id
                JOIN Empresa ON Despacho.empresa_id = Empresa.id AND Empresa.vigente = TRUE
                JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
                JOIN Plato ON ContenidoPedido.plato_id = Plato.id
                JOIN Usuario ON Pedido.usuario_id = Usuario.id
                FULL OUTER JOIN Suscripcion ON Usuario.id = Suscripcion.usuario_id AND Suscripcion.empresa_id = Empresa.id
                AND Pedido.estado IN ('entregado a cliente', 'cliente cancela', 'delivery cancela', 'restaurante cancela')
                GROUP BY Pedido.id, Empresa.precio_despacho, Empresa.vigente, suscripcion.ciclo, suscripcion.fecha_ultimo_pago, usuario.nombre, usuario.email) as gasto_pedido_comida
            ON Pedido.id = gasto_pedido_comida.pedido_id
            JOIN Usuario ON Pedido.usuario_id = Usuario.id
            GROUP BY Pedido.id, usuario.nombre, usuario.email, Pedido.estado, gasto_pedido_comida.precio_comida, gasto_pedido_comida.precio_despacho
            ORDER BY Pedido.id"""
    query= """SELECT Pedido.id, Usuario.nombre, Usuario.email, Pedido.estado
            FROM Pedido
            JOIN Usuario ON Pedido.usuario_id = Usuario.id
            WHERE Pedido.estado IN ('entregado a cliente', 'Cliente Cancela', 'delivery cancela', 'restaurant cancela')
            
            ORDER BY Pedido.id"""
    return query

#Dado un estilo de plato ingresado por el usuario (string) , muestre todas los platos con ese tipo, los restaurantes que las ofrecen y las opciones de delivery.

def consulta4():
    query = """SELECT Plato.nombre, Res_em.id_restaurant, Res_em.nombre_restaurant, Res_em.sucursal, Res_em.nombre_empresa
        FROM Plato
        JOIN PlatoRestaurant ON Plato.id = PlatoRestaurant.plato_id
        JOIN (SELECT Restaurant.id as id_restaurant, Restaurant.nombre as nombre_restaurant, Restaurant.sucursal, Empresa.nombre as nombre_empresa
                    FROM Restaurant
                    JOIN ConvenioEmpresaRestaurant ON Restaurant.id = ConvenioEmpresaRestaurant.restaurant_id
                    JOIN Empresa ON Empresa.id = ConvenioEmpresaRestaurant.empresa_id
                    WHERE Restaurant.vigente = TRUE) AS Res_em ON PlatoRestaurant.restaurant_id = Res_em.id_restaurant
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.disponibilidad = TRUE"""
    
    return query

#Dado un estilo de plato ingresado (string) por el usuario, muestre todas las platos que pertenezcan a ese estilo y sus restricciones
def consulta5():
    query = """SELECT Restaurant. id, Restaurant.nombre, Restaurant.sucursal, Plato.nombre, Plato.restriccion
        FROM Plato
        JOIN Platorestaurant ON Plato.id = PlatoRestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.estilo = %(estilo)s 
        AND Plato.disponibilidad = TRUE
        AND Restaurant.vigente = TRUE"""
    return query

#Dado un cliente ingresado por el usuario (email string), muestre todas las restaurantes a las que tiene acceso con sus suscripciones.

def consulta6():
    #query devuelve nombre del restaurante y nombre de la empresa
    query = """SELECT DISTINCT Restaurant.nombre as nombre_restaurant, Restaurant.sucursal, Restaurant.direccion, Empresa.nombre as nombre_empresa, Suscripcion.ciclo as tipo_suscripcion
        FROM Restaurant 
        JOIN ConvenioEmpresaRestaurant ON Restaurant.id = ConvenioEmpresaRestaurant.restaurant_id
        JOIN Empresa ON Empresa.id = ConvenioEmpresaRestaurant.empresa_id
        JOIN Suscripcion ON Empresa.id = Suscripcion.empresa_id
        JOIN Usuario ON Suscripcion.usuario_id = Usuario.id
        WHERE Usuario.email = %(email)s
        AND Restaurant.vigente = TRUE
        AND Empresa.vigente = TRUE
        AND ( (Suscripcion.ciclo = 'anual' AND Suscripcion.fecha_ultimo_pago >= (NOW() - INTERVAL '1' YEAR) )  
        OR  (Suscripcion.ciclo = 'mensual' AND Suscripcion.fecha_ultimo_pago >= (NOW() - INTERVAL '1' MONTH) ) )
        ORDER BY nombre_restaurant, sucursal, nombre_empresa, tipo_suscripcion"""
    return query

# Muestre la suma de dinero gastada por cada cliente en pedidos no incluidas en planes de suscripcion

def consulta7():
    query = """SELECT Usuario.email, (SUM(gasto_comida.precio_comida) + SUM(gasto_comida.precio_despacho)) as gasto_total
        FROM Usuario, Suscripcion, Pedido 
        (SELECT Pedido.id as pedido_id, Pedido.usuario_id, Empresa.id as empresa_id, Empresa.precio_despacho, SUM(Plato.precio * ContenidoPedido.cantidad) as precio_comida 
            FROM Pedido
            JOIN Despacho ON Despacho.pedido_id = Pedido.id
            JOIN Empresa ON Despacho.empresa_id = Empresa.id
            JOIN ContenidoPedido ON ContenidoPedido.pedido_id = Pedido.id
            JOIN Plato ON ContenidoPedido.plato_id = Plato.id
            WHERE Pedido.estado = 'entregado a cliente'  
            GROUP BY Pedido.id, Pedido.usuario_id, Empresa.id, Empresa.precio_despacho) as gasto_pedido
        WHERE Usuario.id = gasto_pedido.usuario_id 
        AND Pedido.id = gasto_pedido.pedido_id 
        AND Suscripcion.usuario_id = Usuario.id 
        AND Suscripcion.empresa_id NOT IN (SELECT empresa_id 
                                        FROM Suscripcion 
                                        WHERE Suscripcion.usuario_id = Usuario.id 
                                        AND Suscripcion.estado = 'Vigente') 
        GROUP BY Usuario.email"""
    return query

#Muestre todos los platos y los restaurantes que los ofrecen

def consulta8():
    query = """SELECT Plato.nombre, Restaurant.id, Restaurant.nombre, Restaurant.sucursal 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = PlatoRestaurant.plato_id 
        JOIN Restaurant ON PlatoRestaurant.restaurant_id = Restaurant.id
        WHERE Plato.disponibilidad = TRUE
        AND Restaurant.vigente = TRUE"""
    return query

#Dado un numero (1-5) ingresado por el usuario, muestre todos las evaluaciones de Clientes, Delivery, Despachador superiores o iguales a  ́el

def consulta9(tipo: str):

    if tipo == 'cliente':
        query = """SELECT Pedido.id, Usuario.nombre as cliente, Usuario.email, Pedido.evaluacion_cliente
        FROM Pedido
        JOIN Usuario ON Pedido.usuario_id = Usuario.id
        WHERE Pedido.evaluacion_cliente >= %(numero)s"""
        return query
    elif tipo == 'delivery':
        query = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_delivery, Empresa.nombre
        FROM Pedido 
        JOIN Despacho ON Pedido.id = Despacho.pedido_id
        JOIN Empresa ON Despacho.empresa_id = Empresa.id
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""
        return query
    elif tipo == 'despachador':
        query = """SELECT Pedido.id, Pedido.evaluacion_servicio as evaluacion_despachador, Despachador.nombre
        FROM Pedido 
        JOIN Serviciodespacho ON Pedido.id = Serviciodespacho.pedido_id
        JOIN Despachador ON Serviciodespacho.despachador_id = Despachador.id
        WHERE Pedido.evaluacion_servicio >= %(numero)s"""
        return query

#Dado una un alergeno ingresado por el usuario (string), muestre todos los platos que lo contengan en sus ingredientes (Ejemplo: maní)
def consulta10():
    query = """SELECT Restaurant.id, Restaurant.nombre as nombre_restaurant, Restaurant.sucursal, Plato.nombre as nombre_plato 
        FROM Plato
        JOIN Platorestaurant ON Plato.id = Platorestaurant.plato_id 
        JOIN Restaurant ON Platorestaurant.restaurant_id = Restaurant.id
        WHERE Plato.ingredientes LIKE %(alergeno)s 
        AND Plato.disponibilidad = TRUE"""
    return query

In [27]:
def consultas(numero_consulta, **kwargs):
    conn = None
    cursor = None
    
    try:
        # Conexión con la base de datos
        conn = psycopg2.connect(
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT,
            dbname=DB_NAME
        )
        # Crear un cursor para ejecutar consultas
        cursor = conn.cursor()

        if numero_consulta == 1:
            try:
                query = consulta1()
                nombre_plato = kwargs.get('nombre_plato')
                cursor.execute(query, {'nombre': nombre_plato})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 2:
            try:
                query, query1 = consulta2()
                email = kwargs.get('email')
                fecha_inicio = kwargs.get('fecha_inicio')
                fecha_fin = kwargs.get('fecha_fin')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
                cursor.execute(query1, {'fecha_inicio': fecha_inicio, 'fecha_fin': fecha_fin})
                rows = cursor.fetchall()
                print(rows)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 3:
            try:
                query = consulta3()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 4:
            try:
                query = consulta4()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 5:
            try:
                query = consulta5()
                estilo = kwargs.get('estilo')
                cursor.execute(query, {'estilo': estilo})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 6:
            try:
                query = consulta6()
                email = kwargs.get('email')
                cursor.execute(query, {'email': email})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 7:
            try:
                query = consulta7()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 8:
            try:
                query = consulta8()
                cursor.execute(query)
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)
        
        if numero_consulta == 9:
            try:
                tipo = kwargs.get('tipo')
                query = consulta9(tipo)
                numero = kwargs.get('numero')
                cursor.execute(query, {'numero': numero})
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

        if numero_consulta == 10:
            try:
                query = consulta10()
                alergeno = kwargs.get('alergeno')
                cursor.execute(query, {'alergeno': '%{}%'.format(alergeno)}) 
                rows = cursor.fetchall()
                for row in rows:
                    print(row)
            except (Exception, Error) as error:
                print("Error al ejecutar query:", error)

    except (Exception, Error) as error:
        print("Error al conectarse al servidor:", error)
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [28]:
consultas(1, nombre_plato='Pizza Margarita')

(162, 'Just Pizza', 'Maintenci\x97n')
(163, 'Just Pizza', 'Grecia')
(164, 'Just Pizza', 'Estado')
(165, 'Just Pizza', 'Apoquindo')
(166, 'Just Pizza', 'Viel')
(167, 'Just Pizza', 'OHiggins')
(168, 'Just Pizza', 'Prat')
(169, 'Just Pizza', 'Alemania')
(170, 'Just Pizza', 'Larga')
(171, 'Just Pizza', 'Col\x97n')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(85, 'Doble Queso', 'Estado')
(86, 'Doble Queso', 'Apoquindo')
(87, 'Doble Queso', 'Viel')
(88, 'Doble Queso', 'OHiggins')
(89, 'Doble Queso', 'Prat')
(90, 'Doble Queso', 'Alemania')
(16, 'Mestiza', 'Lientur')


In [29]:
consultas(3)

(2, 'Nicolas Crisosto', 'Nicolas.Crisosto@bbdduc.utopia', 'entregado a cliente')
(7, 'Nicolas Zambrano', 'Nicolas.Zambrano@bbdduc.utopia', 'entregado a cliente')
(8, 'Andres Balzan', 'Andres.Balzan@bbdduc.utopia', 'restaurant cancela')
(9, 'Diego Fernandez', 'Diego.Fernandez@bbdduc.utopia', 'restaurant cancela')
(10, 'Angeles Monzon', 'Angeles.Monzon@bbdduc.utopia', 'entregado a cliente')
(11, 'Jose Dauber', 'Jose.Dauber@bbdduc.utopia', 'entregado a cliente')
(13, 'Agustin Matte', 'Agustin.Matte@bbdduc.utopia', 'restaurant cancela')
(15, 'Ignacio Mellado', 'Ignacio.Mellado@bbdduc.utopia', 'entregado a cliente')
(16, 'Ignacia Rolando', 'Ignacia.Rolando@bbdduc.utopia', 'delivery cancela')
(19, 'Andrea Guevara', 'Andrea.Guevara@bbdduc.utopia', 'entregado a cliente')
(22, 'Ignacio Conejeros', 'Ignacio.Conejeros@bbdduc.utopia', 'restaurant cancela')
(23, 'Ignacia Greene', 'Ignacia.Greene@bbdduc.utopia', 'entregado a cliente')
(24, 'Nicolas Munoz', 'Nicolas.Munoz@bbdduc.utopia', 'entregado a

In [ ]:
consultas(4, estilo='Internacional')

In [ ]:
consultas(5, estilo='Internacional')

In [ ]:
consultas(6, email='Eduardo.Osorio@bbdduc.utopia' )

In [ ]:
consultas(8)

In [ ]:
consultas(9, numero = 4, tipo = 'delivery')

In [ ]:
consultas(10, alergeno = 'albahaca')